## Travel Time Simulation for Distance + Open hours (Scenario 2)
* Author: Callie Clark
* Last Updated: 6/8/2024

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import networkx as nx
import osmnx as ox
import json

%matplotlib inline
ox.__version__

from scipy.spatial import cKDTree
from TT_functions import *
import random
from haversine import haversine, Unit

# Calculate TT

In [34]:
census_tract=pd.read_csv('geo_files/census_tract_centroid.csv',index_col=0)
census_tract.set_index(keys='GEOID',inplace=True)
CT_gpd = gpd.GeoDataFrame(
    census_tract, geometry=gpd.points_from_xy(census_tract.lon, census_tract.lat))
CT_gpd=CT_gpd.set_crs(2263)  #use projected crs for distance calculation
CT_gpd.head()

,BoroName,NTA2020,lon,lat,centroid,geometry
GEOID,,,,,,
36061000100,Manhattan,MN0191,-74.042340,40.695673,POINT (-74.04233991488555 40.69567298240604),POINT (-74.042 40.696)
36061000201,Manhattan,MN0301,-73.985848,40.711869,POINT (-73.98584800028144 40.71186871871548),POINT (-73.986 40.712)
36061000600,Manhattan,MN0301,-73.989477,40.711951,POINT (-73.9894767752886 40.711950582311424),POINT (-73.989 40.712)
36061001401,Manhattan,MN0302,-73.987332,40.715160,POINT (-73.98733213277733 40.715160475095416),POINT (-73.987 40.715)
36061001402,Manhattan,MN0302,-73.985882,40.717513,POINT (-73.98588163638996 40.71751329107455),POINT (-73.986 40.718)


In [5]:
fp_df=gpd.read_file("FP_data/food_pantry_schedule_final.geojson",index_col=0)
#fp_df.set_index('id',inplace=True)
fp_df=fp_df.set_crs(2263,allow_override=True)
fp_df.head(n=1)


,id,Name,Address,Hours FINAL,Hours FPC,Hours Plentiful,lat,lng,Additional Notes,BoroName,...,Th_close,Fr_open,Fr_close,Sa_open,Sa_close,Su_open,Su_close,Hours_parsed,hours_open,geometry
0,84142,St. Albans Baptist Church,196-20 119th Ave,"Sa 10:00AM-12:00PM ""every first and third Satu...","Sa 10:00AM-12:00PM ""every first and third Satu...",NaT,40.690698,-73.753923,- Will not refuse anyone even if they don't ha...,Queens,...,NaN,NaN,NaN,10.0,12.0,NaN,NaN,1.0,2.0,POINT (-73.754 40.691)


In [55]:
tract_shp = gpd.read_file('geo_files/nyct2020.shp')
nyc_CT=tract_shp[tract_shp.BoroName.isin(['Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'])]         
#nyc_CT=nyc_CT.to_crs(4326)
nyc_CT.set_index('GEOID',inplace=True)
nyc_CT.index=nyc_CT.index.astype('int')

In [64]:

def make_tt_df_1D(fp_df,CT_gpd):

    """
    Function calls all relevant function to format a result df of travel times and nearest nodes

    Parameters
    ----------
    G(osmnx graph): pass in osmnx graph with adjusted travel times
    fp_df(df): fp dataframe
    census_df(df):census tract df


     Returns
    -------
    df_tt(df): df with all CT nodes in index and two columns fore each time period labeld as "day_TOD_nearest_node" and "day_TOD_tt"

    """
  
    open_fp_dict=get_time_table_id(fp_df.set_index('id'))


    for time_period in open_fp_dict.keys(): #add loop where all is evaluated


        fp_node=list(open_fp_dict[time_period])

        fp_df_open=fp_df.loc[fp_df.id.isin(fp_node)]#index fp_df based on open FPs
        idx_nearest=ckdnearest(CT_gpd, fp_df_open,1)

        idx_nearest_3=ckdnearest(CT_gpd, fp_df_open,k_nearest=3)#tt for top 3
        dist_haver=[]
        dist_haver_3=[]
        
        for i in range(len(idx_nearest_3)):
            dist_haver_iter=[]
            for j in range(3):
                dist_haver_iter.append(haversine((CT_gpd.iloc[i].lat,CT_gpd.iloc[i].lon),(fp_df_open.iloc[idx_nearest_3[i][j]].lat,fp_df_open.iloc[idx_nearest_3[i][j]].lng), unit=Unit.MILES))

            dist_haver_3.append(np.mean(dist_haver_iter))

            dist_haver.append(np.min(dist_haver_iter))



        CT_gpd[time_period+'_nearest']=fp_df_open.iloc[idx_nearest].id.values


        CT_gpd[time_period+'_tt']=[round(i*(60/2.8),2) for i in dist_haver] #takes miles and applies walking speed to get distance in minutes 
        
        CT_gpd[time_period+'_tt_3']=[round(i*(60/2.8),2) for i in dist_haver_3]
       
    return CT_gpd, open_fp_dict



In [ ]:
df, open_fp_dict=make_tt_df_1D(fp_df,CT_gpd)
df.head()

In [68]:
df.drop(columns=['BoroName', 'NTA2020', 'lon', 'lat', 'centroid', 'geometry']).to_csv('travel_time_df/df_tt_S2.csv')
